In [3]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
#from simple_salesforce import Salesforce
#import shutil
#import datetime
#from datetime import date
import json, re
#from pathlib import Path
#import pandas as pd
#import numpy as np
#from io import StringIO
import shutil
import datetime
from datetime import datetime,date
#import re
#import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()


""" sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
) """

' sf = Salesforce(\n    username="jmartinez@careassistance.com",\n    password="Jcmm01041983*",\n    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",\n) '

In [4]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [5]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [15]:
# Clinica_deporte

query_deportes = "SELECT Id,Name,CreatedDate,Canal__c, recordTypeId, Seleccione__c, Seleccione2__c,  Seleccione4__c from Ficha_Cl_nica_Deporte__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_deportes = sf.smart_query(query_deportes)
df_deportes.shape
df_deportes["Seleccione__c"] = df_deportes["Seleccione__c"].astype(str).replace("Se intenta contactar para realizar ingreso, sin éxito.", "1")
df_deportes["Seleccione2__c"] = df_deportes["Seleccione2__c"].astype(str).replace("Se intenta contactar una vez para realizar seguimiento, sin éxito.", "1")
df_deportes["Seleccione2__c"] = df_deportes["Seleccione2__c"].astype(str).replace("Se intenta contactar dos veces para realizar seguimiento, sin éxito.", "1")
df_deportes["Seleccione4__c"] = df_deportes["Seleccione4__c"].astype(str).replace("No Contesta", "1")
df_deportes["Realizado"] = np.where((df_deportes["Seleccione__c"] != "1") & (df_deportes["Seleccione2__c"]!= "1") & (df_deportes["Seleccione4__c"]!="1"),"SI","NO")
condlist = [
    df_deportes["Canal__c"] == "Chatter",
    df_deportes["Canal__c"] == "Llamada",
    df_deportes["Canal__c"] == "Videollamada",
    df_deportes["Canal__c"] == "Correo electrónico",
]
choicelist = [
    "Derivación",
    "Atención Agendada",
    "Atención Agendada",
    "Complemento a la atención",
]
df_deportes["Tipo"] = np.select(condlist, choicelist)

df_deportes["Clínica"] = "Deporte"




100%|██████████| 1/1 [00:02<00:00,  2.97s/it]


In [13]:
query_nutricion = "SELECT Id,Name,CreatedDate,Canal__c, recordTypeId,Seleccione5__c from Ficha_Clinica_Nutrici_n__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 ORDER BY CreatedDate"
df_nutricion = sf.smart_query(query_nutricion)
df_nutricion["Seleccione5__c"] = (
    df_nutricion["Seleccione5__c"].astype(str).replace("No Contesta", "1")
)
df_nutricion["Realizado"] = np.where(
    (df_nutricion["Seleccione5__c"] != "1"), "SI", "NO"
)
condlist = [
    df_nutricion["Canal__c"] == "Chatter",
    df_nutricion["Canal__c"] == "Llamada",
    df_nutricion["Canal__c"] == "Videollamada",
    df_nutricion["Canal__c"] == "Correo electrónico",
]
choicelist = [
    "Derivación",
    "Atención Agendada",
    "Atención Agendada",
    "Complemento a la atención",
]
df_nutricion["Tipo"] = np.select(condlist, choicelist)
df_nutricion["Clínica"] = "Nutrición"


df_nutricion.head()

100%|██████████| 1/1 [00:08<00:00,  8.34s/it]


,Id,Name,CreatedDate,Canal__c,Seleccione5__c,Realizado,Tipo,Clínica
0,a1XRN000000DgMP2A0,FCN-0016441,1704199015000,Llamada,-,SI,Atención Agendada,Nutrición
1,a1XRN000000DgO12AK,FCN-0016442,1704201719000,Videollamada,-,SI,Atención Agendada,Nutrición
2,a1XRN000000DgRF2A0,FCN-0016443,1704201990000,Llamada,1,NO,Atención Agendada,Nutrición
3,a1XRN000000DgW52AK,FCN-0016444,1704203342000,Videollamada,-,SI,Atención Agendada,Nutrición
4,a1XRN000000DgZJ2A0,FCN-0016445,1704203620000,Llamada,-,SI,Atención Agendada,Nutrición


In [16]:
df_deportes.head(50)

,Id,Name,CreatedDate,Canal__c,Seleccione__c,Seleccione2__c,Seleccione4__c,Realizado,Tipo,Clínica
0,a1bRN000000WmCnYAK,FCD-7118,1704197510000,Llamada,-,-,-,SI,Atención Agendada,Deporte
1,a1bRN000000WmEPYA0,FCD-7119,1704197931000,Llamada,-,-,-,SI,Atención Agendada,Deporte
2,a1bRN000000WmfpYAC,FCD-7120,1704199491000,Llamada,-,-,-,SI,Atención Agendada,Deporte
3,a1bRN000000WmpVYAS,FCD-7121,1704201197000,Llamada,-,-,-,SI,Atención Agendada,Deporte
4,a1bRN000000Wmr7YAC,FCD-7122,1704201852000,Llamada,-,-,Se Realiza,SI,Atención Agendada,Deporte
5,a1bRN000000WmsjYAC,FCD-7123,1704201899000,Llamada,-,-,-,SI,Atención Agendada,Deporte
6,a1bRN000000Wo6XYAS,FCD-7124,1704204432000,Videollamada,1,-,-,NO,Atención Agendada,Deporte
7,a1bRN000000Wpc5YAC,FCD-7125,1704206573000,Videollamada,-,-,-,SI,Atención Agendada,Deporte
8,a1bRN000000WyFZYA0,FCD-7126,1704228117000,Videollamada,-,-,-,SI,Atención Agendada,Deporte
9,a1bRN000000X3n7YAC,FCD-7127,1704283963000,Videollamada,-,-,-,SI,Atención Agendada,Deporte
